In [16]:
# This file plays the mpc motion only in viz
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pinocchio as pin
import crocoddyl
from matplotlib import pyplot as plt

from py_biconvex_mpc.motion_planner.cpp_biconvex import BiConvexMP
from inverse_kinematics_cpp import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator
from robot_properties_solo.config import Solo12Config

import raisimpy as raisim
import subprocess
from py_biconvex_mpc.bullet_utils.solo_mpc_env import Solo12Env

from mpc_cartwheel import CarthwheelGen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
robot = Solo12Config.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

pin_robot = Solo12Config.buildRobotWrapper()
pin_robot = Solo12Config.buildRobotWrapper()
urdf = Solo12Config.urdf_path


q0 = np.array(Solo12Config.initial_configuration)
v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


In [58]:
viz.viewer.jupyter_cell()

In [204]:
sim_t = 0.0
sim_dt = .001
index = 0
pln_ctr = 0
plan_freq = 0.1 # sec
update_time = 0.0 # sec (time of lag)
lag = int(update_time/sim_dt)

q = q0
v = v0

cartwheel = CarthwheelGen(q0, v0)

for o in range(20):
    contact_configuration = np.array([1,1,1,1])
    xs, us, f = cartwheel.generate_plan(q, v, sim_t)
    xs = xs[lag:]
    us = us[lag:]
    f = f[lag:]

    time.sleep(0.001)
    for ind in range(int(plan_freq/sim_dt)):
        viz.display(xs[ind][:robot.model.nq])

    q = xs[int(plan_freq/sim_dt)-1][0:pin_robot.model.nq]
    v = xs[int(plan_freq/sim_dt)-1][pin_robot.model.nq:]

    sim_t += plan_freq
#     sim_t = np.round(sim_t%1.0, 3)

Maximum iterations reached 
Final norm: 0.00385877
dyn : 0.053544044494628906
ik : 0.029384851455688477
total : 0.08292984962463379


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

Maximum iterations reached 
Final norm: 0.00965571
dyn : 0.048879146575927734
ik : 0.044707536697387695
total : 0.09358906745910645


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

Maximum iterations reached 
Final norm: 0.107045
dyn : 0.0718846321105957
ik : 0.11174988746643066
total : 0.1836378574371338
Maximum iterations reached 
Final norm: 0.10806
dyn : 0.05092263221740723
ik : 0.020566225051879883
total : 0.07149195671081543


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

ValueError: negative dimensions are not allowed